# 分词
分词不仅限于使用str.split()的方法。而是需要将标点符号分开、将缩写还原成全拼、用正则化表达式的工具将意义相近的词合并在一起（词干还原）：去ing


In [42]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
sentence2 = "Kaggle is the world’s largest data science community with powerful tools and resources to help you achieve your data science goals."
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [4]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


对句子进行切分的最简单的方法就是利用空白

# 独热向量

In [5]:
import numpy as np
token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))
','.join(vocab)

'26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the'

In [7]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)
onehot_vectors = np.zeros((num_tokens, vocab_size), int)
onehot_vectors

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [10]:
for i, word in enumerate(token_sequence):
    onehot_vectors[i, vocab.index(word)] = 1

In [11]:
''.join(vocab)

'26.JeffersonMonticelloThomasageatbeganbuildingofthe'

In [12]:
onehot_vectors

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [13]:
import pandas as pd
pd.DataFrame(onehot_vectors, columns = vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


# 构建词袋向量的DataFrame



Pandas Series 字典

In [28]:
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,twenty-six.
sent,1,1,1,1,1,1,1,1,1,1


In [32]:
sentences = "Construction was done mostly by local masons and carpenters.\n" \
    "He moved into the South Pavilion in 1770.\n" \
    "Turning Monticello into a neoclassical masterpiece in the Palladian style was his perennial project.\n"

corpus = {}
for i, sent in enumerate(sentences.split('\n')):
    corpus['sent{}'.format(i)] = dict([(token, 1) for token in sent.split()])
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df

,Construction,was,done,mostly,by,local,masons,and,carpenters.,He,...,Turning,Monticello,a,neoclassical,masterpiece,Palladian,style,his,perennial,project.
sent0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
sent2,0,1,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
sent3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 度量词袋之间的重合度

In [35]:
df = df.T
df.sent0.dot(df.sent1)

0

In [36]:
df.sent0.dot(df.sent2)

1

结果表明sent0和sent2之间有一个重合的词

# 标点符号的处理

In [43]:
import re
tokens = re.split(r'[-\s.,;!?]+', sentence)
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '']

# NLTK分词

In [44]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+/\S+')
tokenizer.tokenize(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

n-gram

In [45]:
from nltk.util import ngrams
list (ngrams(tokens,2))

[('Thomas', 'Jefferson'),
 ('Jefferson', 'began'),
 ('began', 'building'),
 ('building', 'Monticello'),
 ('Monticello', 'at'),
 ('at', 'the'),
 ('the', 'age'),
 ('age', 'of'),
 ('of', '26'),
 ('26', '')]

# 缩写和网络用语的分词处理
casual_tokenize

In [53]:
from nltk.tokenize.casual import casual_tokenize
message = """RT @TJMonticello Best day everrrrrrr at Monticello. Awesommmmmmeeeeeeee day  I wasn't unhappy :* )"""
print(casual_tokenize(message))

print(casual_tokenize(message, reduce_len=True, strip_handles=True))

['RT', '@TJMonticello', 'Best', 'day', 'everrrrrrr', 'at', 'Monticello', '.', 'Awesommmmmmeeeeeeee', 'day', 'I', "wasn't", 'unhappy', ':', '*', ')']
['RT', 'Best', 'day', 'everrr', 'at', 'Monticello', '.', 'Awesommmeee', 'day', 'I', "wasn't", 'unhappy', ':', '*', ')']


# 停用词处理

In [55]:
stop_words = ['a', 'an','the','on','of','off','this','is']
test_sentence = "The house is on fire"
test_token = casual_tokenize(test_sentence)
test_token

['The', 'house', 'is', 'on', 'fire']

In [58]:
test_token_without_stopwords = [x for x in test_token if x not in stop_words]
test_token_without_stopwords

['The', 'house', 'fire']

# NLTK停用词表

In [60]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\69016\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [62]:
test_token_without_stopwords_1 = [x for x in tokens if x not in stop_words]
test_token_without_stopwords_1

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'age',
 '26',
 '']

# 词汇表归一化

# 大小写转换

In [63]:
tokens = ['House', 'Visitor','Center']
normalized_tokens = [x.lower() for x in tokens]
normalized_tokens

['house', 'visitor', 'center']

# 词干还原

In [65]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
new_sentence = "dish washer's washed dishes"

In [68]:
y = ' '.join([stemmer.stem(w).strip("'") for w in new_sentence.split()])

In [70]:
test_token = casual_tokenize(y)
test_token

['dish', 'washer', 'wash', 'dish']